In [ ]:
!pip install datasets transformers[sentencepiece]  sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 36.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay

In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-hi-en"

In [ ]:
# raw_datasets = load_dataset("cfilt/iitb-english-hindi")
# from google.colab import drive
# drive.mount('/content/drive')

import pandas as pd
from datasets import Dataset, DatasetDict
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data.csv')

tdf = data[0: 2500] # training data frame
vdf = data[2500:] # validation data frame

tds = Dataset.from_pandas(tdf)
vds = Dataset.from_pandas(vdf)

ds = DatasetDict()
ds['train'] = tds
ds['validation'] = vds
print(ds)

DatasetDict({
    train: Dataset({
        features: ['Hindi_Text', 'English_Text'],
        num_rows: 2500
    })
    validation: Dataset({
        features: ['Hindi_Text', 'English_Text'],
        num_rows: 369
    })
})


In [ ]:
raw_datasets = load_dataset("acloudfan/hindi-to-english-translate")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/97 [00:00<?, ? examples/s]

In [ ]:
type(raw_datasets), type(ds)

In [ ]:
ds, raw_datasets

In [ ]:
# ds['train']['Hindi_Text'].extend(raw_datasets['train']['hindi'])
# ds['train']['English_Text'].extend(raw_datasets['train']['english'])

In [ ]:
len(ds['train']), len(ds['validation'])

#### Preprocessing the data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
print(model_checkpoint)

Helsinki-NLP/opus-mt-hi-en


In [ ]:
tokenizer(data['Hindi_Text'][0]), data['Hindi_Text'][0]

({'input_ids': [21291, 58, 0], 'attention_mask': [1, 1, 1]}, 'बचाओ!')

In [ ]:
with tokenizer.as_target_tokenizer():
  print(tokenizer(data['English_Text'][0]), data['English_Text'][0])

{'input_ids': [3862, 58, 0], 'attention_mask': [1, 1, 1]} Help!


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
max_input_length = 128
max_target_length = 128

source_lang = "Hindi_Text"
target_lang = "English_Text"

def preprocess_function(examples):
  inputs = [ex for ex in examples[source_lang]]
  targets = [ex for ex in examples[target_lang]]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

  model_inputs['labels'] = labels['input_ids']
  return model_inputs


In [ ]:
tokenized_datasets = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/369 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Hindi_Text', 'English_Text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2500
    })
    validation: Dataset({
        features: ['Hindi_Text', 'English_Text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 369
    })
})

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/305M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-hi-en.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-hi-en.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
for i in range(50):
  input_text = data['Hindi_Text'][i]
  print(input_text) # 1
  # input_text = 'अरे! मुझे भारत के बारे में बताओ'

  tokenized = tokenizer([input_text], return_tensors='np')
  out = model.generate(**tokenized, max_length=128)
  # print(out)
  with tokenizer.as_target_tokenizer():
    decoded_output = tokenizer.decode(out[0], skip_special_tokens=True)
  print(decoded_output) # 2
  print(data['English_Text'][i]) # 3


बचाओ!
Help!
Help!
उछलो.
Jump.
Jump.
कूदो.
Jump.
Jump.
छलांग.
Consumption.
Jump.
नमस्ते।
Hi.
Hello!
नमस्कार।
Hello.
Hello!
वाह-वाह!
Whoo-ah!
Cheers!
चियर्स!
Chits!
Cheers!
समझे कि नहीं?
Don't you understand that?
Got it?
मैं ठीक हूँ।


KeyboardInterrupt: 

In [ ]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors= 'tf')

In [ ]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors='tf', pad_to_multiple_of=128)

In [ ]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets['train'],
    batch_size=batch_size,
    shuffle=True,
    collate_fn = data_collator,
)

In [ ]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets['validation'],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [ ]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets['validation'],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [ ]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# Unfreeze only the last few layers
for layer in model.layers[-3:]:  # Unfreeze the last three layers
    layer.trainable = True

In [ ]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=5)

Epoch 1/5
156/156 [==============================] - 106s 285ms/step - loss: 0.1106 - accuracy: 0.6710 - val_loss: 1.6797 - val_accuracy: 0.5491
Epoch 2/5
156/156 [==============================] - 29s 189ms/step - loss: 0.0887 - accuracy: 0.6750 - val_loss: 1.7294 - val_accuracy: 0.5497
Epoch 3/5
156/156 [==============================] - 29s 188ms/step - loss: 0.0745 - accuracy: 0.6765 - val_loss: 1.7588 - val_accuracy: 0.5484
Epoch 4/5
156/156 [==============================] - 29s 187ms/step - loss: 0.0644 - accuracy: 0.6766 - val_loss: 1.8102 - val_accuracy: 0.5484
Epoch 5/5
156/156 [==============================] - 29s 188ms/step - loss: 0.0564 - accuracy: 0.6800 - val_loss: 1.8341 - val_accuracy: 0.5436


In [ ]:
model_checkpoint2 = '/content/drive/My Drive/Colab Notebooks/Translation/translation_model2' # 5 epoch
model.save_pretrained(model_checkpoint2)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[61126]], 'forced_eos_token_id': 0}


In [ ]:
# model_checkpoint2 = '/content/drive/My Drive/Colab Notebooks/Translation/translation_model' # 1 epoch
# model.save_pretrained(model_checkpoint2)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[61126]], 'forced_eos_token_id': 0}


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at /content/drive/My Drive/Colab Notebooks/Translation/translation_model2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
for i in range(50):
  input_text = data['Hindi_Text'][i]
  print(input_text) # 1
  # input_text = 'अरे! मुझे भारत के बारे में बताओ'

  tokenized = tokenizer([input_text], return_tensors='np')
  out = model.generate(**tokenized, max_length=128)
  # print(out)
  with tokenizer.as_target_tokenizer():
    decoded_output = tokenizer.decode(out[0], skip_special_tokens=True)
  print(decoded_output) # 2
  print(data['English_Text'][i]) # 3


बचाओ!


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Help!
Help!
उछलो.
Jump.
Jump.
कूदो.
Jump.
Jump.
छलांग.
Slip.
Jump.
नमस्ते।
Hello.
Hello!
नमस्कार।
Hello.
Hello!
वाह-वाह!
Whoa!
Cheers!
चियर्स!
Chits!
Cheers!
समझे कि नहीं?
Don't you understand that?
Got it?
मैं ठीक हूँ।
I'm fine.
I'm OK.
बहुत बढ़िया!
Excellent!
Awesome!
अंदर आ जाओ।
Come in.
Come in.
बाहर निकल जाओ!
Get out!
Get out!
चले जाओ!
Go away!
Go away!
ख़ुदा हाफ़िज़।
May God protect you.
Goodbye!
उत्तम!
Best!
Perfect!
सही!
Correct!
Perfect!
आपका स्वागत है।
Welcome.
Welcome.
स्वागतम्।
Welcome.
Welcome.
मज़े करना।
Have fun.
Have fun.
मौज करना।
Have a good time.
Have fun.
मज़े करो।
Have fun.
Have fun.
मैं भूल गया।
I forgot.
I forgot.
मैं भूल गई।
I forgot.
I forgot.
मैं पैसे दूंगा।
I'll pay.
I'll pay.
मैं ठीक हूँ।
I'm fine.
I'm fine.
मेरा पेट भर गया है।
My stomach is full.
I'm full.
चलो चलें!
Let's go!
Let's go!
मुझे जवाब दो।
Answer me.
Answer me.
पंछी उड़ते हैं।
Birds fly.
Birds fly.
माफ़ कीजिए।
Excuse me.
Excuse me.
बहुत ख़ूब!
Very much!
Fantastic!
खेद की बात है, लेकिन वैसा ही है।


In [ ]:
model_checkpoint2 = '/content/drive/My Drive/Colab Notebooks/Translation/translation_model2'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint2)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at /content/drive/My Drive/Colab Notebooks/Translation/translation_model2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
for i in range(500):
  input_text = data['Hindi_Text'][i]
  print(input_text) # 1
  # input_text = 'अरे! मुझे भारत के बारे में बताओ'

  tokenized = tokenizer([input_text], return_tensors='np')
  out = model.generate(**tokenized, max_length=128)
  # print(out)
  with tokenizer.as_target_tokenizer():
    decoded_output = tokenizer.decode(out[0], skip_special_tokens=True)
  print(decoded_output) # 2
  print(data['English_Text'][i]) # 3


बचाओ!
Help!
Help!
उछलो.
Jump.
Jump.
कूदो.
Jump.
Jump.
छलांग.
Jump.
Jump.
नमस्ते।
Hello!
Hello!
नमस्कार।
Hello!
Hello!
वाह-वाह!
Cheers!
Cheers!
चियर्स!
Cheers!
Cheers!
समझे कि नहीं?
Got it?
Got it?
मैं ठीक हूँ।
I'm OK.
I'm OK.
बहुत बढ़िया!
Wonderful!
Awesome!
अंदर आ जाओ।
Come in.
Come in.
बाहर निकल जाओ!
Get out!
Get out!
चले जाओ!
Go away!
Go away!
ख़ुदा हाफ़िज़।
Goodbye!
Goodbye!
उत्तम!
Perfect!
Perfect!
सही!
Perfect!
Perfect!
आपका स्वागत है।
Welcome.
Welcome.
स्वागतम्।
Welcome.
Welcome.
मज़े करना।
Have fun.
Have fun.
मौज करना।
Have fun.
Have fun.
मज़े करो।
Have fun.
Have fun.
मैं भूल गया।
I forgot.
I forgot.
मैं भूल गई।
I forgot.
I forgot.
मैं पैसे दूंगा।
I'll pay.
I'll pay.
मैं ठीक हूँ।
I'm OK.
I'm fine.
मेरा पेट भर गया है।
I'm full.
I'm full.
चलो चलें!
Let's go!
Let's go!
मुझे जवाब दो।
Answer me.
Answer me.
पंछी उड़ते हैं।
Birds fly.
Birds fly.
माफ़ कीजिए।
Excuse me.
Excuse me.
बहुत ख़ूब!
Fantastic!
Fantastic!
खेद की बात है, लेकिन वैसा ही है।
I fear so.
I fear so.
मैं हँसा।
I laughed

KeyboardInterrupt: 

In [ ]:
# Transformer model- https://www.youtube.com/watch?v=5ToW5Hpi8Qc

### Fine Tuning
https://www.youtube.com/watch?v=WWcgHjuKVqA&t=1291s <br>
https://www.youtube.com/watch?v=eC6Hd1hFvos <br>
https://www.youtube.com/watch?v=V1-Hm2rNkik <br>

### Translation output evaluation:
https://kvashee.medium.com/understanding-mt-quality-bleu-scores-9a19ed20526d <br>
https://towardsdatascience.com/evaluating-text-output-in-nlp-bleu-at-your-own-risk-e8609665a213 <br>
https://www.rws.com/blog/understanding-mt-quality-bleu-scores/ <br>
https://arxiv.org/ftp/arxiv/papers/1601/1601.02789.pdf <br>
https://ai.stackexchange.com/questions/35296/what-are-the-differences-between-bleu-and-meteor <br>

## Research papers about the topic **neural machine translation using "fine-tuning" "transfer learning"**
https://link.springer.com/article/10.1007/s13369-022-06588-w <br>
Other papers in local directory, link given below=> <br>

https://openreview.net/pdf?id=2-80CZSK1eh <br>
https://aclanthology.org/D17-1156.pdf <br>
https://aclanthology.org/2018.iwslt-1.8.pdf <br>
https://web.stanford.edu/class/cs224n/final-reports/final-report-169508363.pdf <br>
https://arxiv.org/pdf/2312.07250.pdf <br>
https://ar5iv.labs.arxiv.org/html/1909.09524 (Baingan) <br>
https://ojs.aaai.org/index.php/AAAI/article/view/6479/6335 <br>
https://www.sciencedirect.com/science/article/pii/S1877050922024899?ref=cra_js_challenge&fr=RR-1 <br>


## Study on **Transfer Learning for NLP**
https://www.analyticsvidhya.com/blog/2020/07/transfer-learning-for-nlp-fine-tuning-bert-for-text-classification/ <br>
https://kedion.medium.com/fine-tuning-nlp-models-with-hugging-face-f92d55949b66 <br>
https://neptune.ai/blog/hugging-face-pre-trained-models-find-the-best <br>
https://spotintelligence.com/2023/03/28/transfer-learning-large-language-models/ (*)<br>

I am studying research papers. You please study the section "Study on Transfer Learning for NLP" and the coding part of transfer learning.



In [ ]:
!pip install sentencepiece
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
model_checkpoint = "Helsinki-NLP/opus-mt-hi-en"

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
model_checkpoint2 = '/content/drive/My Drive/Colab Notebooks/Translation/translation_model2'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint2)

In [ ]:
text = "बचाओ!"
tokenized_text = tokenizer(text, return_tensors="pt")
print(tokenized_text)

{'input_ids': tensor([[21291,    58,     0]]), 'attention_mask': tensor([[1, 1, 1]])}


In [ ]:
translation = model.generate(**tokenized_text)
translated_text = tokenizer.batch_decode(translation, skip_special_tokens=True)[0]
print(translated_text)

In [ ]:
def translate(input_text):
    tokenized_text = tokenizer(input_text, return_tensors="pt")
    translation = model.generate(**tokenized_text)
    translated_text = tokenizer.batch_decode(translation, skip_special_tokens=True)[0]
    return translated_text

#### BLEU

In [ ]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
nltk.download('punkt')

def calculate_bleu(reference, candidate):
    reference_token_list = reference['input_ids'].tolist()
    candidate_token_list = candidate['input_ids'].tolist()

    # Calculate BLEU score
    bleu_score = corpus_bleu([reference_token_list], candidate_token_list)
    return bleu_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def find_bleu_score(data, english_pred):
  n = len(data)
  score = []
  for i in range(n):
      hindi_text = data['Hindi_Text'][i]
      english_actual_text = data['English_Text'][i]

      # Translate the tokenized Hindi text
      # english_pred = translate(hindi_text)
      eng_pred = english_pred[i]

      # Tokenize the predicted English text
      tokenized_english_pred = tokenizer(eng_pred, return_tensors="pt")

      # Tokenize the actual English text
      tokenized_english_actual = tokenizer(english_actual_text, return_tensors="pt")

      # Calculate BLEU score
      sc = calculate_bleu(tokenized_english_actual, tokenized_english_pred)

      score.append((hindi_text, english_actual_text, eng_pred, sc))
  return score

#### ROUGE

In [ ]:
!pip install rouge_score
from rouge_score import rouge_scorer

def calculate_rogue(reference, candidate):
    reference_text = reference
    candidate_text = candidate

    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Calculate ROUGE scores
    rouge_scores = scorer.score(reference_text, candidate_text)

    return rouge_scores

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=810407c9866c0cb32826fb102fc0860b22768021cd70e139f03fb227ca60712e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from string import punctuation
from nltk.tokenize import word_tokenize

def combine_rouge_scores(rouge1, rouge_l):
    combined_score = (rouge1 + rouge_l) / 2.0
    return combined_score

def find_rouge_score(data, english_pred):
  score = []
  n = len(data)

  # Customized list of punctuation characters excluding hyphen
  custom_punctuation = ''.join([char for char in punctuation if char != '-'])

  for i in range(n):
      hindi_text = data['Hindi_Text'][i]
      english_actual_text = data['English_Text'][i]

      # Tokenize and remove punctuation for English actual text
      english_tokens_actual = word_tokenize(english_actual_text)
      english_tokens_no_punct_actual = [word for word in english_tokens_actual if word.isalnum() or word.strip(custom_punctuation)]
      english_actual_text = ' '.join(english_tokens_no_punct_actual)

      # Translate the tokenized Hindi text
      # english_pred = translate(hindi_text)
      eng_pred = english_pred[i]

      # Tokenize and remove punctuation for predicted English text
      english_tokens_pred = word_tokenize(eng_pred)
      english_tokens_no_punct_pred = [word for word in english_tokens_pred if word.isalnum() or word.strip(custom_punctuation)]
      eng_pred = ' '.join(english_tokens_no_punct_pred)

      # Calculate BLEU score
      sc = calculate_rogue(english_actual_text, eng_pred)

      # Extract integer values from the ROUGE scores
      rouge1_integer = int(sc['rouge1'].precision * 100)  # Multiply by 100 to convert to integer
      rougeL_integer = int(sc['rougeL'].precision * 100)
      sc = combine_rouge_scores(rouge1_integer, rougeL_integer)

      score.append((hindi_text, english_actual_text, eng_pred, sc))
  return score

#### METEOR

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation

nltk.download('wordnet')
nltk.download('punkt')

from nltk.translate import meteor_score

# Customized list of punctuation characters excluding hyphen
custom_punctuation = ''.join([char for char in punctuation if char != '-'])

def find_meteor_score(data, english_pred):
  score = []
  n = len(data)

  for i in range(n):
      hindi_text = data['Hindi_Text'][i]
      english_actual_text = data['English_Text'][i]

      # Tokenize and remove punctuation for English actual text
      english_tokens_actual = word_tokenize(english_actual_text)
      english_tokens_no_punct_actual = [word for word in english_tokens_actual if word.isalnum() or word.strip(custom_punctuation)]
      english_actual_text = ' '.join(english_tokens_no_punct_actual)

      # Translate the tokenized Hindi text
      # english_pred = translate(hindi_text)
      eng_pred = english_pred[i]

      # Tokenize and remove punctuation for predicted English text
      english_tokens_pred = word_tokenize(eng_pred)
      english_tokens_no_punct_pred = [word for word in english_tokens_pred if word.isalnum() or word.strip(custom_punctuation)]
      eng_pred = ' '.join(english_tokens_no_punct_pred)

      # Tokenize the predicted English text
      tokenized_english_pred = tokenizer(eng_pred, return_tensors="pt", padding=True, truncation=True)

      # Tokenize the actual English text
      tokenized_english_actual = tokenizer(english_actual_text, return_tensors="pt", padding=True, truncation=True)

      # Extract the tokens from the tokenized tensors
      tokens_pred = tokenized_english_pred["input_ids"].squeeze().tolist()
      tokens_actual = tokenized_english_actual["input_ids"].squeeze().tolist()

      # Convert token IDs back to strings
      tokens_pred_str = tokenizer.convert_ids_to_tokens(tokens_pred)
      tokens_actual_str = tokenizer.convert_ids_to_tokens(tokens_actual)

      # Calculate METEOR score
      sc = meteor_score.meteor_score([tokens_actual_str], tokens_pred_str)

      score.append((hindi_text, english_actual_text, eng_pred, sc))
  return score

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### **Find a good benchmark dataset**
https://huggingface.co/datasets/acloudfan/hindi-to-english-translate <br>
https://huggingface.co/datasets/Aarif1430/english-to-hindi?row=14 <br>
https://huggingface.co/datasets/damerajee/english-to-hindi-l <br>
https://huggingface.co/datasets/hind_encorp


### **Create a pipeline to be used.**

In [ ]:
!pip install datasets transformers[sentencepiece]  sacrebleu -q

In [ ]:
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM
import pandas as pd

model_checkpoint = "Helsinki-NLP/opus-mt-hi-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint2)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at /content/drive/My Drive/Colab Notebooks/Translation/translation_model2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
len(data)

2869

In [ ]:
df = pd.DataFrame()

# Translating the sentences by the model
decoded_outputs = []
n = len(data)

for i in range(n):
  print(i)
  input_text = data['Hindi_Text'][i]
  # print(input_text) # 1

  tokenized = tokenizer([input_text], return_tensors='np')
  out = model.generate(**tokenized, max_length=128)
  # print(out)
  with tokenizer.as_target_tokenizer():
    decoded_output = tokenizer.decode(out[0], skip_special_tokens=True)
  # print(decoded_output) # 2
  # print(data['English_Text'][i]) # 3
  decoded_outputs.append(decoded_output)

  # Create a DataFrame with the updated decoded outputs
  df = pd.DataFrame({'Decoded_Output': decoded_outputs})

  # Save the DataFrame to a CSV file
  df.to_csv('/content/drive/MyDrive/Colab Notebooks/Translation/decoded_outputs1.csv', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Translation/decoded_outputs1.csv')

# Extract the column containing the decoded outputs as a list
decoded_outputs = df['Decoded_Output'].tolist()

In [ ]:
bs = find_bleu_score(data[:n], decoded_outputs)

In [ ]:
df['BLEU_SCORE'] = [i[3] for i in bs]

In [ ]:
rs = find_rouge_score(data[:n], decoded_outputs)

In [ ]:
df['ROUGE_SCORE'] = [i[3] for i in rs]

In [ ]:
ms = find_meteor_score(data[:n], decoded_outputs)

In [ ]:
df['METEOR_SCORE'] = [i[3] for i in ms]

In [ ]:
df

,BLEU_SCORE,ROUGE_SCORE,METEOR_SCORE
0,1.000000e+00,100.0,0.981481
1,1.000000e+00,100.0,0.981481
2,1.000000e+00,100.0,0.981481
3,7.579654e-155,0.0,0.156250
4,1.384293e-231,0.0,0.250000
5,1.646211e-231,100.0,0.937500
6,6.968148e-155,0.0,0.119048
7,9.283143e-155,0.0,0.250000
8,6.373704e-155,0.0,0.217391
9,6.313993e-78,66.0,0.735294


In [ ]:
dfnew = pd.DataFrame()

In [ ]:
df['BLEU_SCORE'].mean(), df['ROUGE_SCORE'].mean(), df['METEOR_SCORE'].mean()

(0.4271214524383409, 65.68473335657023, 0.6463260174571945)

In [ ]:
dfnew['BLEU_SCORE'] = [df['BLEU_SCORE'].mean()]
dfnew['ROUGE_SCORE'] = [df['ROUGE_SCORE'].mean()/100]
dfnew['METEOR_SCORE'] = [df['METEOR_SCORE'].mean()]

dfnew

,BLEU_SCORE,ROUGE_SCORE,METEOR_SCORE
0,0.427121,0.656847,0.646326


In [ ]:
row_names = ['After fine-tuning']
dfnew.index = row_names

In [ ]:
dfnew

,BLEU_SCORE,ROUGE_SCORE,METEOR_SCORE
After fine-tuning,0.427121,0.656847,0.646326


In [ ]:
# Path to save CSV file in Google Drive
file_path = '/content/drive/My Drive/Colab Notebooks/Translation/results.csv'

# Save DataFrame to CSV
dfnew.to_csv(file_path, index=False)

## **Gradual Unfreezing**

In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Translation/data.csv')

### Pretrained model with encoder frozen over all epochs

In [ ]:
from transformers import MarianMTModel, MarianTokenizer, AdamWeightDecay
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load pretrained model and tokenizer
model_checkpoint = "Helsinki-NLP/opus-mt-hi-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# # Preprocess data
# train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)
# test_data = data

# Freeze the encoder layers
for layer in model.model.encoder.layers:
    layer.trainable = False

# Compile the model
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# Fit the model
model.fit(train_dataset, validation_data=validation_dataset, epochs=6)

Epoch 1/6
156/156 [==============================] - 1107s 6s/step - loss: 1.1716 - accuracy: 0.5169 - val_loss: 1.4307 - val_accuracy: 0.5525
Epoch 2/6
156/156 [==============================] - 958s 6s/step - loss: 0.9432 - accuracy: 0.5398 - val_loss: 1.4144 - val_accuracy: 0.5553
Epoch 3/6
156/156 [==============================] - 930s 6s/step - loss: 0.7823 - accuracy: 0.5563 - val_loss: 1.4172 - val_accuracy: 0.5583
Epoch 4/6
156/156 [==============================] - 962s 6s/step - loss: 0.6699 - accuracy: 0.5726 - val_loss: 1.4323 - val_accuracy: 0.5568
Epoch 5/6
156/156 [==============================] - 961s 6s/step - loss: 0.5736 - accuracy: 0.5890 - val_loss: 1.4476 - val_accuracy: 0.5550
Epoch 6/6
154/156 [============================>.] - ETA: 11s - loss: 0.4867 - accuracy: 0.6055

In [ ]:
model_checkpoint4 = '/content/drive/My Drive/Colab Notebooks/Translation/translation_encoder_frozen_model' # 6 epoch
model.save_pretrained(model_checkpoint4)

### Pretrained model with gradual unfreezing

In [ ]:
from transformers import MarianMTModel, MarianTokenizer, AdamWeightDecay
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load pretrained model and tokenizer
model_checkpoint = "Helsinki-NLP/opus-mt-hi-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# # Preprocess data
# train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)
# test_data = data

# Freeze the encoder layers
for layer in model.model.encoder.layers:
    layer.trainable = False

# Compile the model
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer, metrics=['accuracy'])

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-hi-en.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
# Define the batch size
batch_size = 8

freeze = 0
# Gradual unfreezing: Freeze encoder for the first half of training epochs
for epoch in range(6):
    if epoch < 3:  # Freeze encoder for the first 3 epochs
        freeze = 1
        for layer in model.model.encoder.layers:
            layer.trainable = False
    else:  # Unfreeze encoder for the remaining epochs
        freeze = 0
        for layer in model.model.encoder.layers:
            layer.trainable = True

    # # Train the model for one epoch
    print(f"Fitting the model epoch {epoch}, freeze {'No' if freeze==0 else 'Yes'}")
    model.fit(train_dataset, validation_data=validation_dataset, epochs=1)


Fitting the model epoch 0, freeze Yes


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
156/156 [==============================] - 148s 305ms/step - loss: 1.1860 - accuracy: 0.5127 - val_loss: 1.4327 - val_accuracy: 0.5517
Fitting the model epoch 1, freeze Yes
156/156 [==============================] - 30s 190ms/step - loss: 0.9317 - accuracy: 0.5412 - val_loss: 1.4169 - val_accuracy: 0.5547
Fitting the model epoch 2, freeze Yes
156/156 [==============================] - 29s 188ms/step - loss: 0.7855 - accuracy: 0.5634 - val_loss: 1.4172 - val_accuracy: 0.5583
Fitting the model epoch 3, freeze No
156/156 [==============================] - 29s 185ms/step - loss: 0.6632 - accuracy: 0.5828 - val_loss: 1.4305 - val_accuracy: 0.5571
Fitting the model epoch 4, freeze No
156/156 [==============================] - 31s 200ms/step - loss: 0.5679 - accuracy: 0.5866 - val_loss: 1.4476 - val_accuracy: 0.5575
Fitting the model epoch 5, freeze No
156/

In [ ]:
model_checkpoint3 = '/content/drive/My Drive/Colab Notebooks/Translation/translation_gradual_unfreezing_model' # 6 epoch
# model.save_pretrained(model_checkpoint3)

In [ ]:
model_checkpoint3 = '/content/drive/My Drive/Colab Notebooks/Translation/translation_gradual_unfreezing_model' # 6 epoch
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint3)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at /content/drive/My Drive/Colab Notebooks/Translation/translation_gradual_unfreezing_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
df = pd.DataFrame()

# Translating the sentences by the model
decoded_outputs = []
n = len(data)

for i in range(n):
  # print(i)
  input_text = data['Hindi_Text'][i]
  # print(input_text) # 1

  tokenized = tokenizer([input_text], return_tensors='np')
  out = model.generate(**tokenized, max_length=128)
  # print(out)
  with tokenizer.as_target_tokenizer():
    decoded_output = tokenizer.decode(out[0], skip_special_tokens=True)
  # print(decoded_output) # 2
  # print(data['English_Text'][i]) # 3
  decoded_outputs.append(decoded_output)

  # Create a DataFrame with the updated decoded outputs
  df = pd.DataFrame({'Decoded_Output': decoded_outputs})

  # Save the DataFrame to a CSV file
  df.to_csv('/content/drive/MyDrive/Colab Notebooks/decoded_outputs2.csv', index=False)


In [ ]:
# import pandas as pd

# # Read the CSV file into a DataFrame
# df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Translation/decoded_outputs1.csv')

# # Extract the column containing the decoded outputs as a list
# decoded_outputs = df['Decoded_Output'].tolist()

In [ ]:
df = pd.DataFrame()

In [ ]:
bs = find_bleu_score(data[:n], decoded_outputs)

In [ ]:
df['BLEU_SCORE'] = [i[3] for i in bs]

In [ ]:
rs = find_rouge_score(data[:n], decoded_outputs)

In [ ]:
df['ROUGE_SCORE'] = [i[3] for i in rs]

In [ ]:
ms = find_meteor_score(data[:n], decoded_outputs)

In [ ]:
df['METEOR_SCORE'] = [i[3] for i in ms]

In [ ]:
df

,BLEU_SCORE,ROUGE_SCORE,METEOR_SCORE
0,1.000000e+00,100.0,0.981481
1,1.000000e+00,100.0,0.981481
2,1.000000e+00,100.0,0.981481
3,1.000000e+00,100.0,0.981481
4,1.221339e-77,100.0,0.937500
...,...,...,...
2864,7.556872e-01,92.0,0.871813
2865,1.404679e-01,37.0,0.389003
2866,4.808832e-01,59.0,0.670729
2867,1.304610e-01,40.5,0.373047


In [ ]:
dfnew2 = pd.DataFrame()

In [ ]:
dfnew2['BLEU_SCORE'] = [df['BLEU_SCORE'].mean()]
dfnew2['ROUGE_SCORE'] = [df['ROUGE_SCORE'].mean()/100]
dfnew2['METEOR_SCORE'] = [df['METEOR_SCORE'].mean()]

In [ ]:
row_names = ['After fine-tuning with gradual unfreezing']
dfnew2.index = row_names

In [ ]:
dfnew2

,BLEU_SCORE,ROUGE_SCORE,METEOR_SCORE
After fine-tuning with gradual unfreezing,0.713147,0.827571,0.82065


In [ ]:
file_path = '/content/drive/My Drive/Colab Notebooks/Translation/results.csv'
dfnew = pd.read_csv(file_path)
dfnew.index = ['After fine-tuning']
dfnew

,BLEU_SCORE,ROUGE_SCORE,METEOR_SCORE
After fine-tuning,0.427121,0.656847,0.646326


In [ ]:
merged_df = pd.concat([dfnew, dfnew2])

In [ ]:
merged_df

,BLEU_SCORE,ROUGE_SCORE,METEOR_SCORE
After fine-tuning,0.427121,0.656847,0.646326
After fine-tuning with gradual unfreezing,0.713147,0.827571,0.820650


In [ ]:
# Path to save CSV file in Google Drive
file_path = '/content/drive/My Drive/Colab Notebooks/Translation/results.csv'

# Save DataFrame to CSV
merged_df.to_csv(file_path, index=True)